<a href="https://colab.research.google.com/github/yusufbaykal/Patika.dev/blob/main/Elma_A%C4%9Fa%C3%A7lar%C4%B1ndaki_Yaprak_Hastal%C4%B1klar%C4%B1n%C4%B1n_Belirlenmesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kütüphanelerin Kurulumu**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import History

**Kaggle**

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files 
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c plant-pathology-2020-fgvc7

**Modeling**

In [ ]:
train_csv = pd.read_csv("/content/train.csv")
train_csv.head()

In [ ]:
test_csv = pd.read_csv("/content/test.csv")
test_csv.head()

In [ ]:
image = plt.imread('//content/drive/MyDrive/images/Train_0.jpg')
plt.imshow(image)
print(image.shape)

In [ ]:
def image_resize(img, size = (None, None), ratio=3):
    if size[0] is None:
        resize_ratio = ratio
        resize_height = int(img.shape[0]/resize_ratio)
        resize_width = int(img.shape[1]/resize_ratio)
        print(f"height: {resize_height}, width: {resize_width}")
    else:
        resize_height = size[0]
        resize_width = size[1]

    img_resize = tf.image.resize(img, [resize_height,resize_width]).numpy()
    img_resize = img_resize.astype(np.uint8)
    return(img_resize)

In [ ]:
plt.figure(1, figsize=(10,10))
plt.subplot(221)
plt.imshow(image_resize(image, ratio = 3))

plt.subplot(222)
plt.imshow(image_resize(image, ratio = 4))
plt.show()

plt.subplot(223)
plt.imshow(image_resize(image, ratio = 5))

plt.subplot(224)
plt.imshow(image_resize(image, ratio = 6))
plt.show()

In [ ]:
img_height = 227
img_width = 341
plt.imshow(image_resize(image, size=(img_height, img_width)))

In [ ]:
train_resized = []

for img_id in train_csv['image_id'].to_list():
    image = plt.imread(f'/content/drive/MyDrive/images/{img_id}.jpg')
    train_resized.append(image_resize(image, (img_height, img_width)))

print(len(train_resized))

test_resized = []

for img_id in test_csv['image_id'].to_list():
    image = plt.imread(f'/content/drive/MyDrive/images/{img_id}.jpg')
    test_resized.append(image_resize(image, (img_height, img_width)))

print(len(test_resized))

In [ ]:
x_train = np.ndarray(shape = (len(train_resized), img_height, img_width, 3), dtype=np.float32)
x_test = np.ndarray(shape = (len(test_resized), img_height, img_width, 3), dtype=np.float32)

for i in range(len(train_resized)):
    x_train[i] = img_to_array(train_resized[i])

for i in range(len(test_resized)):
    x_test[i] = img_to_array(test_resized[i])

x_train = x_train/255
x_test = x_test/255

print(x_train.shape)
print(x_test.shape)

In [ ]:
y_train = train_csv.iloc[:,1:]
y_train.head()

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.applications import InceptionResNetV2

resnet = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

model = Sequential()
model.add(resnet)
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.layers[0].trainable = False

model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics='accuracy')

In [ ]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(x_train, y_train, test_size = 0.2)

print(train_x.shape)
print(train_y.shape)

print(val_x.shape)
print(val_y.shape)

In [ ]:
ımagedata = ImageDataGenerator(rotation_range=25,
                             shear_range=.20,
                             zoom_range=.20,
                             width_shift_range=.20,
                             height_shift_range=.20,
                             horizontal_flip=True,
                             vertical_flip=True
                            )


batch_size = 24
ımagedata_without_aug = ImageDataGenerator()

train_ımagedata = ımagedata_without_aug.flow(train_x, train_y, batch_size=batch_size)

val_ımagedata = ımagedata_without_aug.flow(val_x, val_y, batch_size=batch_size)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit_generator(train_ımagedata, 
                              epochs=8,
                              steps_per_epoch=train_x.shape[0]//batch_size,
                              validation_data = val_ımagedata,
                              validation_steps = val_x.shape[0]//batch_size,
                              callbacks = [callback]
                   )

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()